In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ffmpeg-python
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00


## DownLoad Howto100m

In [ ]:
%cd /content/drive/MyDrive/24w_deep_daiv/data/data/how2100m
!wget https://www.rocq.inria.fr/cluster-willow/amiech/howto100m/HowTo100M.zip

import os
file_name = "HowTo100M.zip"
output_dir = "/content/drive/MyDrive/24w_deep_daiv/data/data/how2100m"
os.system("unzip "+file_name+" -d "+output_dir)

/content/drive/MyDrive/24w_deep_daiv/data/data/how2100m
--2024-03-12 12:36:03--  https://www.rocq.inria.fr/cluster-willow/amiech/howto100m/HowTo100M.zip
Resolving www.rocq.inria.fr (www.rocq.inria.fr)... 128.93.96.7
Connecting to www.rocq.inria.fr (www.rocq.inria.fr)|128.93.96.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1993427357 (1.9G) [application/zip]
Saving to: ‘HowTo100M.zip’

HowTo100M.zip       100%[===================>]   1.86G  13.5MB/s    in 2m 30s  

2024-03-12 12:38:34 (12.7 MB/s) - ‘HowTo100M.zip’ saved [1993427357/1993427357]



0

In [ ]:
import pandas as pd
pd.read_csv('/content/drive/MyDrive/24w_deep_daiv/data/data/how2100m/HowTo100M_v1.csv').tail()

,video_id,category_1,category_2,rank,task_id
1238906,tVc0qvCeVfQ,Hobbies and Crafts,Crafts,41,38941
1238907,gIURSGIXX0Y,Hobbies and Crafts,Crafts,11,37466
1238908,gHTVgMEpR2Q,Food and Entertaining,Recipes,1,101069
1238909,_97kyZVWVG0,Hobbies and Crafts,Crafts,34,119814
1238910,gkjnR3-ZVts,Food and Entertaining,Drinks,6,25938


In [ ]:
os.getcwd()

'/content/drive/MyDrive/24w_deep_daiv/data/data/how2100m'

In [ ]:
import json
import os
import csv
import datasets


_CITATION = """
@inproceedings{miech19howto100m,
   title={How{T}o100{M}: {L}earning a {T}ext-{V}ideo {E}mbedding by {W}atching {H}undred {M}illion {N}arrated {V}ideo {C}lips},
   author={Miech, Antoine and Zhukov, Dimitri and Alayrac, Jean-Baptiste and Tapaswi, Makarand and Laptev, Ivan and Sivic, Josef},
   booktitle={ICCV},
   year={2019},
}
"""

_DESCRIPTION = """\
HowTo100M is a large-scale dataset of narrated videos with an emphasis on instructional videos where content creators teach complex tasks with an explicit intention of explaining the visual content on screen. HowTo100M features a total of
- 136M video clips with captions sourced from 1.2M YouTube videos (15 years of video)
- 23k activities from domains such as cooking, hand crafting, personal care, gardening or fitness
Each video is associated with a narration available as subtitles automatically downloaded from YouTube.
"""

_URL_BASE = "https://www.rocq.inria.fr/cluster-willow/amiech/howto100m/"


class HowTo100M(datasets.GeneratorBasedBuilder):
    """HowTo100M is a large-scale dataset of narrated videos with an emphasis on instructional videos where content creators teach complex tasks with an explicit intention of explaining the visual content on screen."""

    def _info(self):
        return datasets.DatasetInfo(
            description=_DESCRIPTION,
            features=datasets.Features(
                {
                    "video_id": datasets.Value("string"),
                    "path": datasets.Value("string"),
                    "category_1": datasets.Value("string"),
                    "category_2": datasets.Value("string"),
                    "rank": datasets.Value("int32"),
                    "task_description": datasets.Value("string"), # Could be a datasets.features.ClassLabel though they are 124'000 of them...
                    "starts": datasets.features.Sequence(datasets.Value("float32")),
                    "ends": datasets.features.Sequence(datasets.Value("float32")),
                    "captions": datasets.features.Sequence(datasets.Value("string"))
                }
            ),
            supervised_keys=None,
            homepage="https://www.di.ens.fr/willow/research/howto100m/",
            citation=_CITATION,
        )

    def _split_generators(self, dl_manager):
        archive = dl_manager.download_and_extract(
            os.path.join(_URL_BASE, "HowTo100M.zip")
        )
        extension_file_path = dl_manager.download_and_extract("./extensions.json")
        return [
            datasets.SplitGenerator(
                name=datasets.Split.TRAIN,
                gen_kwargs={"files_path": archive, "extension_file_path": extension_file_path},
            ),
        ]

    def _generate_examples(self, files_path, extension_file_path):
        """This function returns the examples."""
        with open(extension_file_path, encoding="utf-8") as json_file:
            extensions = json.load(json_file)

        task_ids_mapping = {}
        with open(os.path.join(files_path, "task_ids.csv"), encoding="utf-8") as csv_file:
            csv_reader = csv.reader(csv_file, delimiter="\t")
            for task_id, task_description in csv_reader:
                task_ids_mapping[int(task_id)] = task_description

        with open(os.path.join(files_path, "caption.json"), encoding="utf-8") as json_file:
            # Json file is 5GB. Will require ~30GB of RAM to load.
            captions = json.load(json_file)

        with open(os.path.join(files_path, "HowTo100M_v1.csv"), encoding="utf-8") as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=",", )
            next(csv_reader) # Skip header
            for idx, (video_id, category_1, category_2, rank, task_id) in enumerate(csv_reader):
                caption_infos = captions[video_id]
                if video_id in extensions:
                    # 41 videos have annotations but no download link. Examples: `EMLwtpuccLM`, `QgBizF_h7wo`, `kU-SDlFX0AM`.
                    path = f"http://howto100m.inria.fr/dataset/{video_id}.{extensions[video_id]}"
                else:
                    path = ""
                yield idx, {
                    "video_id": video_id,
                    "path": path,
                    "category_1": category_1,
                    "category_2": category_2,
                    "rank": int(rank),
                    "task_description": task_ids_mapping[int(task_id)],
                    "starts": caption_infos["start"],
                    "ends": caption_infos["end"],
                    "captions": caption_infos["text"],
                }


## DownLoad Video

In [ ]:
%cd /content/drive/MyDrive/24w_deep_daiv/howto100m_video

/content/drive/MyDrive/24w_deep_daiv/howto100m_video


In [ ]:
!wget --user pej010918 --password pej09180429! -i howto100m_videos.txt

--2024-03-12 13:46:08--  http://howto100m.inria.fr/dataset/NFsc2pCdBAI.mp4
Resolving howto100m.inria.fr (howto100m.inria.fr)... 128.93.102.10
Connecting to howto100m.inria.fr (howto100m.inria.fr)|128.93.102.10|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://howto100m.inria.fr/dataset/NFsc2pCdBAI.mp4 [following]
--2024-03-12 13:46:08--  https://howto100m.inria.fr/dataset/NFsc2pCdBAI.mp4
Connecting to howto100m.inria.fr (howto100m.inria.fr)|128.93.102.10|:443... connected.
HTTP request sent, awaiting response... 401 Unauthorized
Authentication selected: Basic realm="Login"
Reusing existing connection to howto100m.inria.fr:443.
HTTP request sent, awaiting response... 401 Unauthorized

Username/Password Authentication Failed.
--2024-03-12 13:46:09--  http://howto100m.inria.fr/dataset/brZmg320Bb8.mp4
Connecting to howto100m.inria.fr (howto100m.inria.fr)|128.93.102.10|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permane

In [ ]:
!python download.py  --video_list_file howto100m_videos.txt --user_name pej2834 --password pej09180429 --num_threads 20

downloaded:h3I9ksqzeOI.mp4
downloaded:NFsc2pCdBAI.mp4
downloaded:fZUa7ycEBQE.mp4
downloaded:MtbRweKNGso.mp4
downloaded:YecEaTlRrgM.mp4
downloaded:q5AedKs1YfM.mp4
downloaded:T86AfLQtX2Y.mp4
downloaded:a0DqWUOqcKw.mp4
downloaded:brZmg320Bb8.mp4
downloaded:RSGPVUqZCj4.mp4


In [ ]:
import os
import shutil
import pandas as pd
video_name=os.listdir('/content/drive/MyDrive/24w_deep_daiv/howto100m_video/videos')
video_names=[]
for i in video_name:
  value=os.path.splitext(i)[0]
  video_names.append(value)
  src='/content/drive/MyDrive/24w_deep_daiv/howto100m_video/videos'
  dst='/content/drive/MyDrive/24w_deep_daiv/howto100m_video/videos/'+value
  os.mkdir(dst)
  filename=value+'.mp4'
  shutil.move(os.path.join(src, filename), os.path.join(dst, filename))


video_names
os.listdir('/content/drive/MyDrive/24w_deep_daiv/howto100m_video/videos')

['NFsc2pCdBAI',
 'YecEaTlRrgM',
 'brZmg320Bb8',
 'h3I9ksqzeOI',
 'RSGPVUqZCj4',
 'a0DqWUOqcKw',
 'MtbRweKNGso',
 'q5AedKs1YfM',
 'fZUa7ycEBQE',
 'T86AfLQtX2Y']

In [ ]:
#절대 경로
path = os.path.dirname(os.path.abspath('/content/drive/MyDrive/24w_deep_daiv/howto100m_video/videos/MtbRweKNGso/MtbRweKNGso.mp4'))
print(path)

/content/drive/MyDrive/24w_deep_daiv/howto100m_video/videos/MtbRweKNGso


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/24w_deep_daiv/data/data/HowTo100M_v1.csv")
df=df[df['video_id'].isin(video_names)]
df

,video_id,category_1,category_2,rank,task_id
221624,MtbRweKNGso,Cars & Other Vehicles,Cars,83,95176
306987,brZmg320Bb8,Hobbies and Crafts,Crafts,74,119819
367217,NFsc2pCdBAI,Hobbies and Crafts,Crafts,196,92778
520680,h3I9ksqzeOI,Food and Entertaining,Recipes,36,72460
661172,fZUa7ycEBQE,Home and Garden,Home Improvements and Repairs,103,73448
806657,a0DqWUOqcKw,Food and Entertaining,Recipes,79,106433
899542,YecEaTlRrgM,Home and Garden,Gardening,120,36539
914833,T86AfLQtX2Y,Home and Garden,Gardening,97,47199
1022134,q5AedKs1YfM,Food and Entertaining,Recipes,39,61937
1144878,RSGPVUqZCj4,Food and Entertaining,Recipes,123,22659


In [ ]:
#path/to/feature/vid_nVbIUDjzWY4.npy

#os.mkdir('/content/drive/MyDrive/24w_deep_daiv/howto100m_video/parsed_video/')
for i in df.index:
  f_p='/content/drive/MyDrive/24w_deep_daiv/howto100m_video/parsed_video/' #feature path
  v_p='/content/drive/MyDrive/24w_deep_daiv/howto100m_video/videos/' #video path
  df.loc[i,'path']=f_p+df.loc[i,'video_id']+'/'+df.loc[i,'video_id']+'.npy'
  #os.mkdir(f_p+df.loc[i,'video_id'])
  df.loc[i,'video_path']=v_p+df.loc[i,'video_id']+'/'+df.loc[i,'video_id']+'.mp4'

df.head()

,video_id,category_1,category_2,rank,task_id,path,video_path
221624,MtbRweKNGso,Cars & Other Vehicles,Cars,83,95176,/content/drive/MyDrive/24w_deep_daiv/howto100m...,/content/drive/MyDrive/24w_deep_daiv/howto100m...
306987,brZmg320Bb8,Hobbies and Crafts,Crafts,74,119819,/content/drive/MyDrive/24w_deep_daiv/howto100m...,/content/drive/MyDrive/24w_deep_daiv/howto100m...
367217,NFsc2pCdBAI,Hobbies and Crafts,Crafts,196,92778,/content/drive/MyDrive/24w_deep_daiv/howto100m...,/content/drive/MyDrive/24w_deep_daiv/howto100m...
520680,h3I9ksqzeOI,Food and Entertaining,Recipes,36,72460,/content/drive/MyDrive/24w_deep_daiv/howto100m...,/content/drive/MyDrive/24w_deep_daiv/howto100m...
661172,fZUa7ycEBQE,Home and Garden,Home Improvements and Repairs,103,73448,/content/drive/MyDrive/24w_deep_daiv/howto100m...,/content/drive/MyDrive/24w_deep_daiv/howto100m...


In [ ]:
df.to_csv('/content/drive/MyDrive/24w_deep_daiv/data/data/howto100m_10.csv',index=False)

In [ ]:
dfdf=pd.read_csv('/content/drive/MyDrive/24w_deep_daiv/data/data/howto100m_10.csv')
dfdf

,video_id,category_1,category_2,rank,task_id,path,video_path
0,MtbRweKNGso,Cars & Other Vehicles,Cars,83,95176,/content/drive/MyDrive/24w_deep_daiv/howto100m...,/content/drive/MyDrive/24w_deep_daiv/howto100m...
1,brZmg320Bb8,Hobbies and Crafts,Crafts,74,119819,/content/drive/MyDrive/24w_deep_daiv/howto100m...,/content/drive/MyDrive/24w_deep_daiv/howto100m...
2,NFsc2pCdBAI,Hobbies and Crafts,Crafts,196,92778,/content/drive/MyDrive/24w_deep_daiv/howto100m...,/content/drive/MyDrive/24w_deep_daiv/howto100m...
3,h3I9ksqzeOI,Food and Entertaining,Recipes,36,72460,/content/drive/MyDrive/24w_deep_daiv/howto100m...,/content/drive/MyDrive/24w_deep_daiv/howto100m...
4,fZUa7ycEBQE,Home and Garden,Home Improvements and Repairs,103,73448,/content/drive/MyDrive/24w_deep_daiv/howto100m...,/content/drive/MyDrive/24w_deep_daiv/howto100m...
5,a0DqWUOqcKw,Food and Entertaining,Recipes,79,106433,/content/drive/MyDrive/24w_deep_daiv/howto100m...,/content/drive/MyDrive/24w_deep_daiv/howto100m...
6,YecEaTlRrgM,Home and Garden,Gardening,120,36539,/content/drive/MyDrive/24w_deep_daiv/howto100m...,/content/drive/MyDrive/24w_deep_daiv/howto100m...
7,T86AfLQtX2Y,Home and Garden,Gardening,97,47199,/content/drive/MyDrive/24w_deep_daiv/howto100m...,/content/drive/MyDrive/24w_deep_daiv/howto100m...
8,q5AedKs1YfM,Food and Entertaining,Recipes,39,61937,/content/drive/MyDrive/24w_deep_daiv/howto100m...,/content/drive/MyDrive/24w_deep_daiv/howto100m...
9,RSGPVUqZCj4,Food and Entertaining,Recipes,123,22659,/content/drive/MyDrive/24w_deep_daiv/howto100m...,/content/drive/MyDrive/24w_deep_daiv/howto100m...


In [ ]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.8 MB/s eta 0:00:00


In [ ]:
from pytube import YouTube
DownloadFolder = '/content/drive/MyDrive/24w_deep_daiv/howto100m_video/videos'

for id in dfdf['video_id']:
  link='https://www.youtube.com/watch?v={}'.format(id)
  video = YouTube(link)
  clip = video.streams.get_highest_resolution()
  clip.download(DownloadFolder+'/'+id)

In [ ]:
os.listdir('/content/drive/MyDrive/24w_deep_daiv/howto100m_video/videos/MtbRweKNGso')[1]

'Gravely drive belt replacement and tension bracket repair.mp4'

In [ ]:
video_names=[]
for i in dfdf['video_id']:
  ll='/content/drive/MyDrive/24w_deep_daiv/howto100m_video/videos/{}'.format(i)
  video_name=os.listdir(ll)[1]
  value=os.path.splitext(video_name)[0]
  video_names.append(value)
video_names

['Gravely drive belt replacement and tension bracket repair',
 'Bowling Pin Snowman  Christmas in July',
 'Minecraft PE  No Yellow Dye',
 'Easy 3 ingredient Peanut Butter Fudge - How To Make Peanut Butter Fudge',
 'Tiny House 57 - Install Outlet Middle of the Wire Run',
 'how to make Swiss Meringue Buttercream',
 'Phoradendron leucarpum American Mistletoe',
 'Diy compost bin',
 'Spicy Korean Chicken Noodle Stir Fry Recipe  Spicy Sauce Noodle Stir Fry',
 'Egg & Omelet Wave - Review']

In [ ]:
for n,i in enumerate(dfdf['video_id']):
  #os.mkdir(f_p+df.loc[i,'video_id'])
  src='/content/drive/MyDrive/24w_deep_daiv/howto100m_video/videos/{}'.format(i)
  src=src+'/'+os.listdir(src)[1]
  dst=dfdf.loc[n,'video_path']
  os.rename(src,dst)

## Video Loader
* 2D & 3D

In [ ]:
import torch as th
from torch.utils.data.sampler import Sampler
import numpy as np

class RandomSequenceSampler(Sampler):

    def __init__(self, n_sample, seq_len):
        self.n_sample = n_sample
        self.seq_len = seq_len

    def _pad_ind(self, ind):
        zeros = np.zeros(self.seq_len - self.n_sample % self.seq_len)
        ind = np.concatenate((ind, zeros))
        return ind

    def __iter__(self):
        idx = np.arange(self.n_sample)
        if self.n_sample % self.seq_len != 0:
            idx = self._pad_ind(idx)
        idx = np.reshape(idx, (-1, self.seq_len))
        np.random.shuffle(idx)
        idx = np.reshape(idx, (-1))
        return iter(idx.astype(int))

    def __len__(self):
        return self.n_sample + (self.seq_len - self.n_sample % self.seq_len)

In [ ]:
import torch as th

class Normalize(object):

    def __init__(self, mean, std):
        self.mean = th.FloatTensor(mean).view(1, 3, 1, 1)
        self.std = th.FloatTensor(std).view(1, 3, 1, 1)

    def __call__(self, tensor):
        tensor = (tensor - self.mean) / (self.std + 1e-8)
        return tensor

class Preprocessing(object):

    def __init__(self, type):
        self.type = type
        if type == '2d':
            self.norm = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        elif type == '3d':
            self.norm = Normalize(mean=[110.6, 103.2, 96.3], std=[1.0, 1.0, 1.0])

    def _zero_pad(self, tensor, size):
        n = size - len(tensor) % size
        if n == size:
            return tensor
        else:
            z = th.zeros(n, tensor.shape[1], tensor.shape[2], tensor.shape[3])
            return th.cat((tensor, z), 0)

    def __call__(self, tensor):
        if self.type == '2d':
            tensor = tensor / 255.0
            tensor = self.norm(tensor)
        elif self.type == '3d':
            tensor = self._zero_pad(tensor, 16)
            tensor = self.norm(tensor)
            tensor = tensor.view(-1, 16, 3, 112, 112)
            tensor = tensor.transpose(1, 2)
        return tensor

In [ ]:
import sys
import torch as th
import torchvision.models as models
from torch import nn


class GlobalAvgPool(nn.Module):
    def __init__(self):
        super(GlobalAvgPool, self).__init__()

    def forward(self, x):
        return th.mean(x, dim=[-2, -1])


def get_model(args):
    assert args.type in ['2d', '3d']
    if args.type == '2d':
        print('Loading 2D-ResNet-152 ...')
        model = models.resnet152(pretrained=True)
        model = nn.Sequential(*list(model.children())[:-2], GlobalAvgPool())
        #model = model.cuda()
    else:
        print('Loading 3D-ResneXt-101 ...')
        model = resnet101(
            num_classes=400,
            shortcut_type='B',
            cardinality=32,
            sample_size=112,
            sample_duration=16,
            last_fc=False)
        #model = model.cuda()
        model_data = th.load(args.resnext101_model_path)
        model.load_state_dict(model_data)

    model.eval()
    print('loaded')
    return model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import math
from functools import partial

__all__ = ['ResNeXt', 'resnet50', 'resnet101']


def conv3x3x3(in_planes, out_planes, stride=1):
    # 3x3x3 convolution with padding
    return nn.Conv3d(in_planes, out_planes, kernel_size=3,
                     stride=stride, padding=1, bias=False)


def downsample_basic_block(x, planes, stride):
    out = F.avg_pool3d(x, kernel_size=1, stride=stride)
    zero_pads = torch.Tensor(out.size(0), planes - out.size(1),
                             out.size(2), out.size(3),
                             out.size(4)).zero_()
    if isinstance(out.data, torch.cuda.FloatTensor):
        zero_pads = zero_pads.cuda()

    out = Variable(torch.cat([out.data, zero_pads], dim=1))

    return out


class ResNeXtBottleneck(nn.Module):
    expansion = 2

    def __init__(self, inplanes, planes, cardinality, stride=1, downsample=None):
        super(ResNeXtBottleneck, self).__init__()
        mid_planes = cardinality * int(planes / 32)
        self.conv1 = nn.Conv3d(inplanes, mid_planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm3d(mid_planes)
        self.conv2 = nn.Conv3d(mid_planes, mid_planes, kernel_size=3, stride=stride,
                               padding=1, groups=cardinality, bias=False)
        self.bn2 = nn.BatchNorm3d(mid_planes)
        self.conv3 = nn.Conv3d(mid_planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm3d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNeXt(nn.Module):

    def __init__(self, block, layers, sample_size, sample_duration, shortcut_type='B', cardinality=32, num_classes=400, last_fc=True):
        self.last_fc = last_fc

        self.inplanes = 64
        super(ResNeXt, self).__init__()
        self.conv1 = nn.Conv3d(3, 64, kernel_size=7, stride=(1, 2, 2),
                               padding=(3, 3, 3), bias=False)
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1)
        self.layer1 = self._make_layer(block, 128, layers[0], shortcut_type, cardinality)
        self.layer2 = self._make_layer(block, 256, layers[1], shortcut_type, cardinality, stride=2)
        self.layer3 = self._make_layer(block, 512, layers[2], shortcut_type, cardinality, stride=2)
        self.layer4 = self._make_layer(block, 1024, layers[3], shortcut_type, cardinality, stride=2)
        last_duration = math.ceil(sample_duration / 16)
        last_size = math.ceil(sample_size / 32)
        self.avgpool = nn.AvgPool3d((last_duration, last_size, last_size), stride=1)
        self.fc = nn.Linear(cardinality * 32 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, shortcut_type, cardinality, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            if shortcut_type == 'A':
                downsample = partial(downsample_basic_block,
                                     planes=planes * block.expansion,
                                     stride=stride)
            else:
                downsample = nn.Sequential(
                    nn.Conv3d(self.inplanes, planes * block.expansion,
                              kernel_size=1, stride=stride, bias=False),
                    nn.BatchNorm3d(planes * block.expansion)
                )

        layers = []
        layers.append(block(self.inplanes, planes, cardinality, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, cardinality))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)

        x = x.view(x.size(0), -1)
        if self.last_fc:
            x = self.fc(x)

        return x

def get_fine_tuning_parameters(model, ft_begin_index):
    if ft_begin_index == 0:
        return model.parameters()

    ft_module_names = []
    for i in range(ft_begin_index, 5):
        ft_module_names.append('layer{}'.format(ft_begin_index))
    ft_module_names.append('fc')

    parameters = []
    for k, v in model.named_parameters():
        for ft_module in ft_module_names:
            if ft_module in k:
                parameters.append({'params': v})
                break
        else:
            parameters.append({'params': v, 'lr': 0.0})

    return parameters

def resnet50(**kwargs):
    """Constructs a ResNet-50 model.
    """
    model = ResNeXt(ResNeXtBottleneck, [3, 4, 6, 3], **kwargs)
    return model

def resnet101(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNeXt(ResNeXtBottleneck, [3, 4, 23, 3], **kwargs)
    return model

def resnet152(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNeXt(ResNeXtBottleneck, [3, 8, 36, 3], **kwargs)
    return model

In [ ]:
import torch as th
from torch.utils.data import Dataset
import pandas as pd
import os
import numpy as np
import ffmpeg


class VideoLoader(Dataset):
    """Pytorch video loader."""

    def __init__(
            self,
            csv,
            framerate=1,
            size=112,
            centercrop=False,
    ):
        """
        Args:
        """
        self.csv = pd.read_csv(csv)
        self.centercrop = centercrop
        self.size = size
        self.framerate = framerate

    def __len__(self):
        return len(self.csv)

    def _get_video_dim(self, video_path):
        probe = ffmpeg.probe(video_path)
        video_stream = next((stream for stream in probe['streams']
                             if stream['codec_type'] == 'video'), None)
        width = int(video_stream['width'])
        height = int(video_stream['height'])
        return height, width

    def _get_output_dim(self, h, w):
        if isinstance(self.size, tuple) and len(self.size) == 2:
            return self.size
        elif h >= w:
            return int(h * self.size / w), self.size
        else:
            return self.size, int(w * self.size / h)

    def __getitem__(self, idx):
        video_path = self.csv['video_path'].values[idx]
        output_file = self.csv['path'].values[idx]

        if not(os.path.isfile(output_file)) and os.path.isfile(video_path):
            print('Decoding video: {}'.format(video_path))
            try:
                h, w = self._get_video_dim(video_path)
            except:
                print('ffprobe failed at: {}'.format(video_path))
                return {'video': th.zeros(1), 'input': video_path,
                        'output': output_file}
            height, width = self._get_output_dim(h, w)
            cmd = (
                ffmpeg
                .input(video_path)
                .filter('fps', fps=self.framerate)
                .filter('scale', width, height)
            )
            if self.centercrop:
                x = int((width - self.size) / 2.0)
                y = int((height - self.size) / 2.0)
                cmd = cmd.crop(x, y, self.size, self.size)
            out, _ = (
                cmd.output('pipe:', format='rawvideo', pix_fmt='rgb24')
                .run(capture_stdout=True, quiet=True)
            )
            if self.centercrop and isinstance(self.size, int):
                height, width = self.size, self.size
            video = np.frombuffer(out, np.uint8).reshape([-1, height, width, 3])
            video = th.from_numpy(video.astype('float32'))
            video = video.permute(0, 3, 1, 2)
        else:
            video = th.zeros(1)

        return {'video': video, 'input': video_path, 'output': output_file}

In [ ]:

#load video
import torch as th
import math
import numpy as np
from torch.utils.data import DataLoader
import torch.nn.functional as F
import easydict

args = easydict.EasyDict({
        'csv': '/content/drive/MyDrive/24w_deep_daiv/data/data/howto100m_10.csv',
        'batch_size': 64,
        'type': '2d',
        'half_precision': 1,
        'num_decoding_thread': 4,
        'l2_normalize': 1,
        'resnext101_model_path': 'model/resnext101.pth'
    })


dataset = VideoLoader(
    args.csv,
    framerate=1 if args.type == '2d' else 24,
    size=224 if args.type == '2d' else 112,
    centercrop=(args.type == '3d'),
)
n_dataset = len(dataset)
sampler = RandomSequenceSampler(n_dataset, 10)
loader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    num_workers=args.num_decoding_thread,
    sampler=sampler if n_dataset > 10 else None,
)
preprocess = Preprocessing(args.type)
model = get_model(args)

with th.no_grad():
    for k, data in enumerate(loader):
        input_file = data['input'][0]
        output_file = data['output'][0]
        if len(data['video'].shape) > 3:
            print('Computing features of video {}/{}: {}'.format(
                k + 1, n_dataset, input_file))
            video = data['video'].squeeze()
            if len(video.shape) == 4:
                video = preprocess(video)
                n_chunk = len(video)
                #features = th.cuda.FloatTensor(n_chunk, 2048).fill_(0)
                features = th.FloatTensor(n_chunk, 2048).fill_(0)
                n_iter = int(math.ceil(n_chunk / float(args.batch_size)))
                for i in range(n_iter):
                    min_ind = i * args.batch_size
                    max_ind = (i + 1) * args.batch_size
                    video_batch = video[min_ind:max_ind]#.cuda()
                    batch_features = model(video_batch)
                    if args.l2_normalize:
                        batch_features = F.normalize(batch_features, dim=1)
                    features[min_ind:max_ind] = batch_features
                features = features.cpu().numpy()
                if args.half_precision:
                    features = features.astype('float16')
                np.save(output_file, features)
        else:
            print('Video {} already processed.'.format(input_file))

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can al

Loading 2D-ResNet-152 ...
loaded
Decoding video: /content/drive/MyDrive/24w_deep_daiv/howto100m_video/videos/MtbRweKNGso/MtbRweKNGso.mp4Decoding video: /content/drive/MyDrive/24w_deep_daiv/howto100m_video/videos/brZmg320Bb8/brZmg320Bb8.mp4Decoding video: /content/drive/MyDrive/24w_deep_daiv/howto100m_video/videos/NFsc2pCdBAI/NFsc2pCdBAI.mp4
Decoding video: /content/drive/MyDrive/24w_deep_daiv/howto100m_video/videos/h3I9ksqzeOI/h3I9ksqzeOI.mp4


Decoding video: /content/drive/MyDrive/24w_deep_daiv/howto100m_video/videos/T86AfLQtX2Y/T86AfLQtX2Y.mp4
Decoding video: /content/drive/MyDrive/24w_deep_daiv/howto100m_video/videos/YecEaTlRrgM/YecEaTlRrgM.mp4
Decoding video: /content/drive/MyDrive/24w_deep_daiv/howto100m_video/videos/fZUa7ycEBQE/fZUa7ycEBQE.mp4
Computing features of video 1/10: /content/drive/MyDrive/24w_deep_daiv/howto100m_video/videos/MtbRweKNGso/MtbRweKNGso.mp4
Decoding video: /content/drive/MyDrive/24w_deep_daiv/howto100m_video/videos/a0DqWUOqcKw/a0DqWUOqcKw.mp4
Decoding vide